In [54]:
from nicegui import ui, app
import re
import os
import sqlite3
import datetime
import pandas as pd
import plotly.graph_objects as go

In [55]:
conn = sqlite3.connect('stock_trading_records.db')
cur = conn.cursor()
def get_all_records():
    all_records = pd.read_sql(
        "select code,stock_name,timestamp,action,price,quantity,amount,fee from stock_trading_record order by timestamp desc", con=conn)
    print(all_records.shape)
    return all_records

df_all_records = get_all_records()
df_all_records

(10, 8)


,code,stock_name,timestamp,action,price,quantity,amount,fee
0,515790,光伏ETF,2024-02-22 10:00:00,BUY,0.811,1500,1216.62,0.12
1,515790,光伏ETF,2023-09-27 11:22:55,BUY,1.000,300,300.10,0.10
2,515790,光伏ETF,2023-09-11 13:45:08,BUY,1.020,1000,1020.10,0.10
3,515790,光伏ETF,2023-09-04 14:07:42,SELL,1.050,1000,1049.89,0.11
4,515790,光伏ETF,2023-09-04 13:44:22,SELL,1.060,1000,1059.89,0.11
5,515790,光伏ETF,2023-08-29 14:53:28,BUY,1.055,1000,1055.11,0.11
6,515790,光伏ETF,2023-08-29 14:00:59,BUY,1.060,1000,1060.11,0.11
7,515790,光伏ETF,2023-08-28 14:16:32,BUY,1.040,2000,2080.21,0.21
8,515790,光伏ETF,2023-08-28 14:12:43,SELL,1.070,2000,2139.79,0.21
9,515790,光伏ETF,2023-07-21 15:44:25,BUY,1.160,2000,2320.23,0.23


In [56]:
def get_overview_table():
    # df=get_all_records()
    df = df_all_records.copy()
    temp = df.groupby(['code', 'stock_name', 'action']).agg(
        {'amount': sum, 'quantity': sum})
    temp.columns = ['total_amount', 'total_quantity']
    temp.reset_index(inplace=True)
    # print(temp)
    temp = temp.sort_values(['code', 'action'], ascending=[True, True])
    temp['stock_quantity'] = temp['total_quantity'] - \
        temp['total_quantity'].shift(-1)
    temp['stock_amount'] = temp['total_amount']-temp['total_amount'].shift(-1)
    temp = temp[temp['action'] == 'BUY']

    for index, row in temp.iterrows():
        if int(row['stock_quantity']) < 0:
            temp.loc[index, 'stock_quantity'] = row['total_quantity']
            temp.loc[index, 'stock_amount'] = row['total_amount']

    temp['stock_cost'] = temp['stock_amount'].apply(
        lambda x: float(x))/temp['stock_quantity']
    
    temp = temp.drop(['action', 'total_quantity', 'total_amount'], axis=1)

    return temp


df_overview = get_overview_table()
df_overview


/tmp/ipykernel_1077053/2992719583.py:4: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  temp = df.groupby(['code', 'stock_name', 'action']).agg(
/tmp/ipykernel_1077053/2992719583.py:4: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  temp = df.groupby(['code', 'stock_name', 'action']).agg(


,code,stock_name,stock_quantity,stock_amount,stock_cost
0,515790,光伏ETF,4800.0,4802.91,1.000606


In [57]:
codes = list(df_overview['code'].values)
codes

['515790']

In [58]:
import akshare as ak
gfetf=ak.fund_etf_spot_em()
print(gfetf[gfetf['代码']=='515790'])

       代码     名称    最新价    涨跌额   涨跌幅        成交量          成交额    开盘价    最高价  \
3  515790  光伏ETF  0.868  0.018  2.12  3625053.0  316289241.0  0.852  0.882   

     最低价    昨收   换手率         流通市值          总市值  
3  0.852  0.85  3.08  10202032806  10202032806  


In [59]:
df=df_overview.set_index('code').join(gfetf.set_index('代码'))
df=df.drop(['stock_name','成交量','成交额','流通市值','总市值'],axis=1)
# convert just columns "a" and "b"
df[["最新价", "涨跌额",'stock_quantity','stock_amount','stock_cost']] = df[["最新价", "涨跌额",'stock_quantity','stock_amount','stock_cost']].apply(pd.to_numeric)
df['current_amount']=df['最新价']*df['stock_quantity']
df['stock_cost']=df['stock_cost'].apply(lambda x: '%.3f'%x)
df['profit']=df['current_amount']-df['stock_amount']
print(df)

        stock_quantity  stock_amount stock_cost     名称    最新价    涨跌额   涨跌幅  \
code                                                                         
515790          4800.0       4802.91      1.001  光伏ETF  0.868  0.018  2.12   

          开盘价    最高价    最低价    昨收   换手率  current_amount  profit  
code                                                             
515790  0.852  0.882  0.852  0.85  3.08          4166.4 -636.51  


In [65]:
# process detailed transaction data
df=df_all_records.set_index('code').join(gfetf.set_index('代码'))
df=df.drop(['stock_name','成交量','成交额','流通市值','总市值','涨跌额','涨跌幅','开盘价','最高价','最低价','昨收','换手率'],axis=1)
df[["price", "quantity",'amount','fee']] = df[["price", "quantity",'amount','fee']].apply(pd.to_numeric)

df['profit']=(df['最新价']-df['price'])*df['quantity']

df['profit']=df['profit'].mask(df['action']=='SELL',df['profit']*(-1))

print(df)


                  timestamp action  price  quantity   amount   fee     名称  \
code                                                                        
515790  2024-02-22 10:00:00    BUY  0.811      1500  1216.62  0.12  光伏ETF   
515790  2023-09-27 11:22:55    BUY  1.000       300   300.10  0.10  光伏ETF   
515790  2023-09-11 13:45:08    BUY  1.020      1000  1020.10  0.10  光伏ETF   
515790  2023-09-04 14:07:42   SELL  1.050      1000  1049.89  0.11  光伏ETF   
515790  2023-09-04 13:44:22   SELL  1.060      1000  1059.89  0.11  光伏ETF   
515790  2023-08-29 14:53:28    BUY  1.055      1000  1055.11  0.11  光伏ETF   
515790  2023-08-29 14:00:59    BUY  1.060      1000  1060.11  0.11  光伏ETF   
515790  2023-08-28 14:16:32    BUY  1.040      2000  2080.21  0.21  光伏ETF   
515790  2023-08-28 14:12:43   SELL  1.070      2000  2139.79  0.21  光伏ETF   
515790  2023-07-21 15:44:25    BUY  1.160      2000  2320.23  0.23  光伏ETF   

          最新价  profit  
code                   
515790  0.868    85.5  
515